# Phase 7: 에러 핸들링, UX, 문서화

**학습 목표:**
- 에러 타입과 메시지 매핑
- 환경변수 검증
- 로딩 상태 표시 (spinner, progress, status)
- 성공/실패 메시지 패턴
- 빈 상태(empty state) 처리
- Docstring과 타입 힌트
- README 작성법

**인터랙티브 데모:**
```bash
uv run streamlit run materials/phase_7_demo.py
```

---
## Part 1: 에러 핸들링

### 1.1 에러 핸들링이란?

프로그램에서 **문제가 발생했을 때 적절히 대응**하는 것입니다.

| 에러 핸들링 없음 | 에러 핸들링 있음 |
|-----------------|----------------|
| 앱 크래시 | 앱 계속 동작 |
| 빨간 에러 메시지 | 친절한 안내 메시지 |
| 사용자 혼란 | 해결 방법 안내 |

### 1.2 에러 메시지 매핑

**에러 코드 → 사용자 친화적 메시지**로 변환합니다.

In [ ]:
# 에러 메시지 매핑
ERROR_MESSAGES = {
    "api_key_invalid": "API 키가 유효하지 않습니다. 설정을 확인해주세요.",
    "rate_limit_exceeded": "요청이 너무 많습니다. 잠시 후 다시 시도해주세요.",
    "network_error": "네트워크 연결을 확인해주세요.",
    "no_results": "검색 결과가 없습니다.",
    "ai_error": "AI 요약 중 오류가 발생했습니다.",
}

def handle_error(error_type: str):
    message = ERROR_MESSAGES.get(error_type, "알 수 없는 오류")
    print(f"에러: {message}")

# 테스트
handle_error("api_key_invalid")
handle_error("unknown_type")

**매핑 사용의 장점:**
- 일관된 메시지
- 유지보수 용이 (한 곳에서 관리)
- 다국어 지원 쉬움

### 1.3 커스텀 에러 클래스

In [ ]:
class AppError(Exception):
    """앱 전용 에러 클래스"""
    def __init__(self, error_type: str):
        self.error_type = error_type
        super().__init__(error_type)

# 사용 예시
def risky_operation(should_fail: bool):
    if should_fail:
        raise AppError("network_error")
    return "성공!"

try:
    result = risky_operation(True)
except AppError as e:
    message = ERROR_MESSAGES.get(e.error_type, "알 수 없는 오류")
    print(f"처리된 에러: {message}")

### 1.4 환경변수 검증

앱 시작 시 **필수 설정이 있는지 확인**합니다.

In [ ]:
import os

class Settings:
    def __init__(self):
        self.TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
        self.GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
        
        # 검증
        missing = []
        if not self.TAVILY_API_KEY:
            missing.append("TAVILY_API_KEY")
        if not self.GEMINI_API_KEY:
            missing.append("GEMINI_API_KEY")
        
        if missing:
            raise ValueError(f"누락된 환경변수: {', '.join(missing)}")

# Streamlit에서 사용
# try:
#     settings = Settings()
# except ValueError as e:
#     st.error(str(e))
#     st.stop()  # 앱 실행 중단

### 1.5 HTTP 상태 코드별 처리

| 코드 | 의미 | 대응 |
|------|------|------|
| 200 | 성공 | 정상 처리 |
| 400 | 잘못된 요청 | 입력값 확인 안내 |
| 401 | 인증 실패 | API 키 확인 안내 |
| 429 | 요청 한도 초과 | 대기 후 재시도 안내 |
| 5xx | 서버 오류 | 재시도 안내 |

In [ ]:
def handle_api_error(error: Exception):
    """API 에러를 분석하여 적절한 AppError로 변환"""
    error_str = str(error).lower()
    
    if "401" in error_str or "unauthorized" in error_str:
        raise AppError("api_key_invalid")
    elif "429" in error_str or "rate" in error_str:
        raise AppError("rate_limit_exceeded")
    elif "timeout" in error_str or "connection" in error_str:
        raise AppError("network_error")
    else:
        raise AppError("server_error")

---
## Part 2: UX (사용자 경험)

### 2.1 UX란?

**사용자 경험** - 사용자가 앱을 사용하면서 느끼는 전체적인 경험입니다.

| 상황 | 나쁜 UX | 좋은 UX |
|------|---------|--------|
| 로딩 중 | 화면이 멈춤 | 스피너 표시 |
| 성공 | 아무 반응 없음 | "검색 완료!" 메시지 |
| 실패 | 빨간 에러 코드 | "다시 시도해주세요" 안내 |
| 빈 상태 | 빈 화면 | "첫 검색을 시작해보세요!" |

### 2.2 Streamlit 로딩 컴포넌트

| 컴포넌트 | 용도 | 코드 |
|----------|------|------|
| `st.spinner()` | 메시지와 함께 스피너 | `with st.spinner("처리 중..."):` |
| `st.progress()` | 진행률 바 | `st.progress(50)` |
| `st.status()` | 단계별 상태 표시 | `with st.status("작업 중"):` |

```python
# 로딩 래퍼 함수
def show_loading(message: str = "처리 중..."):
    return st.spinner(message)

# 사용
with show_loading("검색 중..."):
    result = search(keyword)
```

### 2.3 메시지 컴포넌트

| 컴포넌트 | 색상 | 용도 |
|----------|------|------|
| `st.success()` | 초록 | 작업 성공 |
| `st.error()` | 빨강 | 오류 발생 |
| `st.warning()` | 노랑 | 주의/경고 |
| `st.info()` | 파랑 | 정보/안내 |
| `st.toast()` | 팝업 | 간단한 알림 |

**좋은 메시지 작성법:**
```python
# X
st.error("Error")

# O
st.error("API 키가 유효하지 않습니다. 설정을 확인해주세요.")
```

메시지에 포함할 정보:
1. **무엇이** 일어났는지
2. **왜** 그런지 (에러의 경우)
3. **어떻게** 해결하는지

### 2.4 빈 상태(Empty State) 처리

데이터가 없을 때의 화면입니다.

```python
# X 나쁜 예
if not results:
    pass  # 아무것도 안 함

# O 좋은 예
if not results:
    st.info("아직 검색 기록이 없습니다.")
    st.markdown("""
    **시작하기:**
    1. 상단 입력창에 키워드를 입력하세요
    2. 검색 버튼을 클릭하세요
    """)
```

빈 상태에 포함할 것:
1. 현재 상태 설명
2. 다음 행동 안내
3. (선택) 예시/팁

### 2.5 TrendTracker의 UX 패턴

```python
# 진행 상태 메시지
with show_loading("뉴스를 검색하고 있습니다..."):
    articles = search_news(keyword)

with show_loading("AI가 요약하고 있습니다..."):
    summary = summarize_news(articles)

# 성공 메시지
st.success(f"'{keyword}' 검색 완료! {len(articles)}건의 뉴스를 분석했습니다.")

# 빈 상태 처리
if not keyword and not search_keys:
    st.info("아직 검색 기록이 없습니다. 키워드를 입력해보세요!")
```

---
## Part 3: 문서화

### 3.1 문서화란?

코드가 **무엇을 하는지 설명**하는 것입니다.

In [ ]:
# X 문서화 없음
def f(x, y, z):
    return [i for i in x if y(i) > z]

In [ ]:
from typing import List, Callable

# O 문서화 있음
def filter_items_by_threshold(
    items: List[int],
    scorer: Callable[[int], float],
    threshold: float
) -> List[int]:
    """
    점수가 임계값을 초과하는 항목만 필터링합니다.

    Args:
        items: 필터링할 항목 리스트
        scorer: 각 항목의 점수를 계산하는 함수
        threshold: 최소 점수 임계값

    Returns:
        점수가 threshold를 초과하는 항목들의 리스트
    """
    return [item for item in items if scorer(item) > threshold]

### 3.2 Docstring 형식

```python
def 함수명(인자: 타입) -> 반환타입:
    """
    함수가 하는 일을 한 줄로 설명합니다.

    Args:
        인자명: 인자에 대한 설명

    Returns:
        반환값에 대한 설명

    Raises:
        발생할 수 있는 예외

    Example:
        >>> 함수명(예시_인자)
        예시_결과
    """
    pass
```

In [ ]:
from typing import List, Optional

def search_news(keyword: str, num_results: int = 5) -> List[dict]:
    """
    키워드로 뉴스를 검색하여 결과를 반환합니다.

    Tavily API를 사용하여 최신 뉴스를 검색하고,
    발행일 기준으로 정렬하여 반환합니다.

    Args:
        keyword: 검색할 키워드
        num_results: 반환할 결과 수 (기본값: 5)

    Returns:
        검색된 뉴스 기사 리스트

    Raises:
        AppError: API 키 오류, 요청 한도 초과 등

    Example:
        >>> articles = search_news("AI", 3)
        >>> len(articles)
        3
    """
    # 구현...
    return []

### 3.3 타입 힌트 (Type Hints)

변수나 함수의 **데이터 타입을 명시**하는 것입니다.

In [ ]:
from typing import Optional, List, Dict, Any

# 변수
name: str = "홍길동"
age: int = 25
is_active: bool = True
scores: List[int] = [90, 85, 88]

# 함수
def greet(name: str) -> str:
    return f"안녕하세요, {name}님!"

def add(a: int, b: int) -> int:
    return a + b

# Optional: 값이 없을 수도 있음
def find_user(user_id: int) -> Optional[dict]:
    return None

# Dict: 딕셔너리
def get_config() -> Dict[str, Any]:
    return {"debug": True, "port": 8080}

**타입 힌트의 장점:**

| 장점 | 설명 |
|------|------|
| 가독성 | 코드만 봐도 타입을 알 수 있음 |
| 에러 방지 | IDE가 타입 오류를 미리 알려줌 |
| 문서화 | 별도 설명 없이도 이해 가능 |
| 자동완성 | IDE가 더 정확한 제안을 함 |

### 3.4 dataclass와 문서화

In [ ]:
from dataclasses import dataclass
from typing import Optional

@dataclass
class NewsArticle:
    """
    뉴스 기사를 나타내는 데이터 클래스입니다.

    Attributes:
        title: 기사 제목
        url: 기사 링크 URL
        snippet: 기사 요약/스니펫
        pub_date: 발행일 (없을 수 있음)
    """
    title: str
    url: str
    snippet: str
    pub_date: Optional[str] = None

# 사용
article = NewsArticle(
    title="AI 뉴스",
    url="https://example.com",
    snippet="AI 관련 소식..."
)
print(article)

### 3.5 README.md 구조

프로젝트의 **첫인상**이자 **사용 설명서**입니다.

```markdown
# TrendTracker

키워드로 뉴스를 검색하고 AI가 요약해주는 웹앱

## 주요 기능
- 최신 뉴스 검색
- AI 핵심 요약
- 검색 기록 관리
- CSV 다운로드

## 설치 방법

### 1. uv 설치
```bash
curl -LsSf https://astral.sh/uv/install.sh | sh
```

### 2. 의존성 설치
```bash
uv sync
```

### 3. 환경변수 설정
```bash
cp .env.example .env
# .env 파일에 API 키 입력
```

### 4. 실행
```bash
uv run streamlit run app.py
```

## API 키 발급
- Tavily: https://tavily.com/
- Gemini: https://aistudio.google.com/
```

---

# ✏️ 가이드 실습: TODO 따라하기

---

## ✏️ 실습 1: 에러 메시지 매핑 만들기

에러 코드를 사용자 친화적 메시지로 변환하는 시스템을 만들어봅시다!

**목표**: ERROR_MESSAGES 딕셔너리와 handle_error 함수 구현

**힌트**:
- 딕셔너리로 에러 타입 → 메시지 매핑
- `dict.get(key, default)` 패턴 사용

In [ ]:
# ============================================
# ✏️ 실습 1: 에러 메시지 매핑 만들기
# ============================================
# 목표: 에러 타입별 사용자 친화적 메시지 시스템 구현

# TODO 1: ERROR_MESSAGES 딕셔너리 만들기
# - "login_failed": "로그인에 실패했습니다. 아이디와 비밀번호를 확인해주세요."
# - "session_expired": "세션이 만료되었습니다. 다시 로그인해주세요."
# - "permission_denied": "접근 권한이 없습니다."


# >>> 정답 (학습 후 주석 해제하여 확인) <<<
# ERROR_MESSAGES = {
#     "login_failed": "🔐 로그인에 실패했습니다. 아이디와 비밀번호를 확인해주세요.",
#     "session_expired": "⏰ 세션이 만료되었습니다. 다시 로그인해주세요.",
#     "permission_denied": "🚫 접근 권한이 없습니다.",
# }

# TODO 2: handle_error 함수 만들기
# - error_type을 받아서 ERROR_MESSAGES에서 메시지 조회
# - 없으면 "알 수 없는 오류가 발생했습니다." 반환
# - Streamlit에서는 st.error(message)로 표시


# >>> 정답 (학습 후 주석 해제하여 확인) <<<
# def handle_error(error_type: str) -> str:
#     """에러 타입에 맞는 사용자 친화적 메시지를 반환합니다."""
#     default = "❓ 알 수 없는 오류가 발생했습니다."
#     return ERROR_MESSAGES.get(error_type, default)

# 시뮬레이션
ERROR_MESSAGES = {
    "login_failed": "🔐 로그인에 실패했습니다. 아이디와 비밀번호를 확인해주세요.",
    "session_expired": "⏰ 세션이 만료되었습니다. 다시 로그인해주세요.",
    "permission_denied": "🚫 접근 권한이 없습니다.",
}

def handle_error(error_type: str) -> str:
    default = "❓ 알 수 없는 오류가 발생했습니다."
    return ERROR_MESSAGES.get(error_type, default)

# 테스트
print("=== 에러 메시지 테스트 ===")
for error_type in ["login_failed", "session_expired", "permission_denied", "unknown"]:
    print(f"{error_type}: {handle_error(error_type)}")

### 💡 실습 1 해설

**TODO 1 해설**: 에러 메시지를 딕셔너리로 정의하면 코드와 메시지를 분리할 수 있습니다. 메시지 수정 시 한 곳만 변경하면 됩니다.

**TODO 2 해설**: `dict.get(key, default)` 패턴으로 없는 키에 대해 기본값을 반환합니다. TrendTracker의 `handle_error()` 함수도 동일한 패턴을 사용합니다.

✅ 잘 따라오셨나요? 다음 실습으로 넘어갑시다!

## ✏️ 실습 2: 빈 상태 처리 구현하기

데이터가 없을 때 사용자에게 친절한 안내를 표시해봅시다!

**목표**: 빈 상태일 때 안내 메시지와 행동 가이드 표시

**힌트**:
- `st.info()`로 현재 상태 설명
- `st.caption()`이나 `st.markdown()`으로 다음 행동 안내

In [ ]:
# ============================================
# ✏️ 실습 2: 빈 상태 처리 구현하기
# ============================================
# 목표: 데이터가 없을 때 친절한 안내 표시

# TODO 1: 빈 상태 메시지 함수 만들기
# - 파라미터: context (어떤 데이터가 비었는지)
# - st.info()로 현재 상태 표시
# - st.caption()으로 다음 행동 안내


# >>> 정답 (학습 후 주석 해제하여 확인) <<<
# def render_empty_state(context: str):
#     """빈 상태일 때 안내 메시지를 표시합니다."""
#     messages = {
#         "search": ("🔍 검색 결과가 없습니다.", "다른 키워드로 검색해보세요!"),
#         "history": ("📋 검색 기록이 없습니다.", "첫 검색을 시작해보세요!"),
#         "favorites": ("⭐ 즐겨찾기가 비어있습니다.", "마음에 드는 기사를 저장해보세요!"),
#     }
#     info, caption = messages.get(context, ("데이터가 없습니다.", ""))
#     st.info(info)
#     if caption:
#         st.caption(caption)

# 시뮬레이션 (Jupyter에서 테스트)
def render_empty_state(context: str):
    """빈 상태일 때 안내 메시지를 표시합니다."""
    messages = {
        "search": ("🔍 검색 결과가 없습니다.", "다른 키워드로 검색해보세요!"),
        "history": ("📋 검색 기록이 없습니다.", "첫 검색을 시작해보세요!"),
        "favorites": ("⭐ 즐겨찾기가 비어있습니다.", "마음에 드는 기사를 저장해보세요!"),
    }
    info, caption = messages.get(context, ("데이터가 없습니다.", ""))
    print(f"[INFO] {info}")
    if caption:
        print(f"  💡 {caption}")

# 테스트
print("=== 빈 상태 메시지 테스트 ===")
for ctx in ["search", "history", "favorites", "unknown"]:
    print(f"\n{ctx}:")
    render_empty_state(ctx)

### 💡 실습 2 해설

**TODO 1 해설**: 빈 상태 메시지도 딕셔너리로 관리하면 일관성 있게 처리할 수 있습니다. 튜플로 (메시지, 안내)를 묶으면 관련 정보를 함께 관리할 수 있습니다.

**핵심 포인트**:
- 빈 화면은 사용자를 혼란스럽게 합니다
- 현재 상태 + 다음 행동 안내가 좋은 UX입니다
- TrendTracker도 검색 기록이 없을 때 안내 메시지를 표시합니다

✅ 실습 2 완료! 이제 챌린지에 도전해보세요!

---

# 🏆 챌린지: 스스로 도전하기

---

## 🏆 챌린지 1: 에러 핸들러 확장

에러 메시지 시스템을 확장해보세요!

**요구사항**:
1. ERROR_MESSAGES에 새로운 에러 타입 3개 이상 추가
2. `handle_error()` 함수에 `level` 파라미터 추가 (error, warning, info)
3. 에러 발생 시 해결 방법도 함께 반환

**힌트**:
- 튜플로 `(메시지, 해결방법)` 저장
- level에 따라 `st.error()`, `st.warning()`, `st.info()` 분기

In [ ]:
# ============================================
# 🏆 챌린지 1: 에러 핸들러 확장
# ============================================
# 아래 코드를 완성해보세요!

from typing import Tuple, Dict

# TODO 1: ERROR_MESSAGES를 확장하세요
# - 메시지와 해결방법을 튜플로 저장
# - 예: "error_type": ("메시지", "해결방법")
ERROR_MESSAGES_EXTENDED: Dict[str, Tuple[str, str]] = {
    # 여기에 최소 5개의 에러 타입을 추가하세요
    # "api_key_invalid": ("API 키가 유효하지 않습니다.", "설정에서 API 키를 확인해주세요."),
}

# TODO 2: handle_error_extended 함수를 구현하세요
def handle_error_extended(error_type: str, level: str = "error"):
    """
    에러 타입과 레벨에 따라 적절한 메시지를 표시합니다.
    
    Args:
        error_type: 에러 유형
        level: 메시지 레벨 ("error", "warning", "info")
    """
    # 여기에 구현하세요
    # 1. ERROR_MESSAGES_EXTENDED에서 메시지, 해결방법 조회
    # 2. level에 따라 다른 방식으로 출력
    pass

# 테스트
# handle_error_extended("api_key_invalid", "error")
# handle_error_extended("no_results", "warning")

## 🏆 챌린지 2: UX 개선 - 로딩 & 피드백

사용자 경험을 개선하는 함수들을 만들어보세요!

**요구사항**:
1. `show_success()` 함수: 성공 메시지 + toast 알림
2. `show_progress()` 함수: 단계별 진행 상태 표시
3. 각 함수에 적절한 docstring과 타입 힌트 추가

**힌트**:
- `st.success()` + `st.toast()` 조합
- `st.status()`로 단계 표시

In [ ]:
# ============================================
# 🏆 챌린지 2: UX 개선 - 로딩 & 피드백
# ============================================
# 아래 코드를 완성해보세요!

from typing import List

# TODO 1: show_success 함수 구현
def show_success(message: str, details: str = ""):
    """
    성공 메시지를 표시하고 toast 알림을 보냅니다.
    
    Args:
        message: 메인 성공 메시지
        details: 추가 상세 정보 (선택)
    """
    # 여기에 구현하세요
    # st.success(message)
    # st.toast(...)
    pass

# TODO 2: show_progress 함수 구현
def show_progress(steps: List[str], title: str = "진행 중..."):
    """
    여러 단계의 진행 상태를 표시합니다.
    
    Args:
        steps: 진행 단계 목록 (예: ["데이터 로드", "처리", "저장"])
        title: 전체 진행 상태 제목
    """
    # 여기에 구현하세요
    # with st.status(title) as status:
    #     for step in steps:
    #         st.write(step)
    #         time.sleep(0.5)
    #     status.update(label="완료!", state="complete")
    pass

# 테스트 (Streamlit 앱에서 실행)
# show_success("검색 완료!", "5건의 뉴스를 찾았습니다.")
# show_progress(["검색 중...", "분석 중...", "요약 중..."], "AI 분석")

## 🏆 챌린지 3: 통합 - 미니 검색 앱

에러 핸들링과 UX를 모두 적용한 미니 검색 앱을 만들어보세요!

**요구사항**:
1. 검색 입력 + 버튼
2. 검색 시 로딩 표시 (spinner)
3. 성공/실패에 따른 메시지 표시
4. 빈 상태 처리 (검색 전, 결과 없음)
5. 에러 발생 시 친절한 안내

**구현할 함수**:
- `validate_input(keyword)`: 입력 검증
- `search_with_feedback(keyword)`: 검색 + UX 피드백
- `render_results(results)`: 결과 표시 또는 빈 상태

**힌트**:
- 이전 챌린지의 함수들을 조합
- try-except로 에러 처리

In [ ]:
# ============================================
# 🏆 챌린지 3: 통합 - 미니 검색 앱
# ============================================
# 아래 코드를 완성해서 미니 검색 앱을 만들어보세요!

from typing import List, Optional

# TODO 1: 입력 검증 함수
def validate_input(keyword: str) -> bool:
    """
    검색어 입력을 검증합니다.
    
    Args:
        keyword: 검색어
        
    Returns:
        유효하면 True, 아니면 False
    """
    # 여기에 구현하세요
    # - 빈 문자열 체크
    # - 최소 길이 체크 (2자 이상)
    pass

# TODO 2: 검색 + UX 피드백 함수
def search_with_feedback(keyword: str) -> Optional[List[dict]]:
    """
    검색을 수행하고 사용자에게 피드백을 제공합니다.
    
    Args:
        keyword: 검색어
        
    Returns:
        검색 결과 리스트 또는 None (에러 시)
    """
    # 여기에 구현하세요
    # 1. validate_input() 호출
    # 2. with st.spinner() 로 로딩 표시
    # 3. try-except로 에러 처리
    # 4. 성공 시 st.success(), 실패 시 handle_error()
    pass

# TODO 3: 결과 렌더링 함수  
def render_results(results: Optional[List[dict]], keyword: str = ""):
    """
    검색 결과를 표시하거나 빈 상태를 처리합니다.
    
    Args:
        results: 검색 결과 리스트
        keyword: 검색어 (빈 상태 메시지용)
    """
    # 여기에 구현하세요
    # - results가 None이면 에러 상태
    # - results가 빈 리스트면 "결과 없음" 표시
    # - results가 있으면 결과 표시
    pass

# Streamlit 앱 구조 예시:
# keyword = st.text_input("검색어")
# if st.button("검색"):
#     results = search_with_feedback(keyword)
#     render_results(results, keyword)

---
## 정리

### 에러 핸들링

```python
# 에러 메시지 매핑
ERROR_MESSAGES = {
    "error_type": "친절한 메시지",
}

# 커스텀 에러
class AppError(Exception):
    def __init__(self, error_type: str):
        self.error_type = error_type

# 에러 처리
try:
    risky_operation()
except AppError as e:
    message = ERROR_MESSAGES.get(e.error_type, "알 수 없는 오류")
    st.error(message)
```

### UX 패턴

```python
# 로딩 표시
with st.spinner("처리 중..."):
    result = long_operation()

# 성공/실패 메시지
if success:
    st.success(f"완료! {count}건 처리")
else:
    st.error("실패. 다시 시도해주세요.")

# 빈 상태 처리
if not items:
    st.info("데이터가 없습니다.")
    st.caption("새 항목을 추가해보세요!")
```

### Docstring 템플릿

```python
def function_name(arg1: Type1, arg2: Type2 = default) -> ReturnType:
    """
    함수가 하는 일을 한 줄로 설명합니다.

    Args:
        arg1: 첫 번째 인자 설명
        arg2: 두 번째 인자 설명 (기본값: default)

    Returns:
        반환값 설명

    Raises:
        ErrorType: 에러 발생 조건
    """
    pass
```

---
## 전체 과정 완료!

| Phase | 주제 |
|-------|------|
| 1 | 프로젝트 초기화 (uv, 환경변수, 구조) |
| 2 | 도메인 모델 (클래스, dataclass, 타입) |
| 3 | 서비스 레이어 (API, 예외 처리) |
| 4 | 리포지토리 (파일 I/O, CSV, pandas) |
| 5 | UI 컴포넌트 (Streamlit) |
| 6 | 메인 앱 통합 |
| 7 | 에러 핸들링 & 마무리 |

**다음 단계:**
1. 실제 프로젝트 실행: `uv run streamlit run app.py`
2. 코드 수정해보기
3. 나만의 프로젝트 시작!